Environment: Google colab with T4 gpu selected.

## Step 1: Train the model for Neutral and NonNeutral.

In [ ]:
# import packages
import pandas as pd

# load google drive
from google.colab import drive
drive.mount('/content/gdrive')

! pip install flair  # install Flair framework

Mounted at /content/gdrive
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 373.1/373.1 kB 7.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 788.5/788.5 kB 58.2 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 135.8/135.8 kB 20.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.7/7.7 MB 108.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 981.5/981.5 kB 71.3 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 53.1/53.1 kB 7.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 19.7/19.7 MB 79.3 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 302.0/302.0 kB 37.6 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━

In [ ]:
# load training data
df = pd.read_csv('/content/gdrive/MyDrive/Enron_email/4763.csv')

In [ ]:
df = df[['Content', 'Label']]  # subset the columns
df.columns = ['Text', 'Sentiment']   # change the column names

In [ ]:
# convert Positive and Negative label to NotNeutral
new_label = []   # create a list to store labels
for index, row in df.iterrows():
  if row['Sentiment'] == 'Neutral':
    new_label.append('Neutral')
  else:
    new_label.append('NotNeutral')
df['Sentiment'] = new_label
df

,Text,Sentiment
0,"Ok, let me know if you got an extra ticket. Se...",Neutral
1,"Done. Sent: Tuesday, November 06, 2001 9:35 AM...",Neutral
2,"Hey, Phillip already made reservations and Jef...",Neutral
3,I probably won't be able to go. Could we send ...,NotNeutral
4,I got this from elizabeth. IF you really want ...,Neutral
...,...,...
4758,This is a summary of my position and trading g...,NotNeutral
4759,"not after today Sent: Tuesday, June 12, 2001 1...",Neutral
4760,Agreement on Power I assume you're going to le...,Neutral
4761,are you on msn messenger. i think i'm set up w...,NotNeutral


In [ ]:
df.to_csv('/content/gdrive/MyDrive/Enron_email/Flair/model1_train/train.csv', index = False)  # save into a csv file for training


In [ ]:
# load modules from Flair framework
from flair.data import Corpus
from flair.embeddings import TransformerDocumentEmbeddings
from flair.datasets import CSVClassificationCorpus
from flair.models import TextClassifier
from flair.trainers import ModelTrainer

data_path = '/content/gdrive/MyDrive/Enron_email/Flair/model1_train' # set a folder to store training data

# Define column names
column_names = {0: 'text', 1: 'label'}

label_type = 'Sentiment'

# Create a Corpus using the CSVClassificationCorpus class
corpus: Corpus = CSVClassificationCorpus(data_folder = data_path,
                                         train_file = '/content/gdrive/MyDrive/Enron_email/Flair/model1_train/train.csv', # specify the training file
                                         column_name_map = {0: 'text', 1: 'label'},
                                         label_type = label_type, skip_header=True)

# create the label dictionary
label_dict = corpus.make_label_dictionary(label_type=label_type)

# initialize transformer document embeddings
document_embeddings = TransformerDocumentEmbeddings('distilbert-base-uncased', fine_tune=True)

# create the text classifier
classifier = TextClassifier(document_embeddings, label_dictionary=label_dict, label_type=label_type)

# initialize trainer
trainer = ModelTrainer(classifier, corpus)

# run training
trainer.train(
    #'/content/gdrive/MyDrive/Enron_email/Flair/model_with_validation',  # Specify the path to save the model
    #'/content/gdrive/MyDrive/Enron_email/Flair/bert_model_final',
    #'/content/gdrive/MyDrive/Enron_email/Flair/model_aug',
    #'/content/gdrive/MyDrive/Enron_email/Flair/model_semi',
    '/content/gdrive/MyDrive/Enron_email/Flair/model1',

    learning_rate=0.01,
    mini_batch_size=32,
    # max_epochs=10,
    max_epochs = 3
)

2023-10-16 12:09:30,603 Reading data from /content/gdrive/MyDrive/Enron_email/Flair/model1_train
2023-10-16 12:09:30,607 Train: /content/gdrive/MyDrive/Enron_email/Flair/model1_train/train.csv
2023-10-16 12:09:30,609 Dev: None
2023-10-16 12:09:30,611 Test: None
2023-10-16 12:09:30,683 Computing label dictionary. Progress:


3858it [00:09, 396.84it/s]

2023-10-16 12:09:40,415 Dictionary created for label 'Sentiment' with 3 values: Neutral (seen 2471 times), NotNeutral (seen 1387 times)


2023-10-16 12:09:46,136 ----------------------------------------------------------------------------------------------------
2023-10-16 12:09:46,139 Model: "TextClassifier(
  (embeddings): TransformerDocumentEmbeddings(
    (model): DistilBertModel(
      (embeddings): Embeddings(
        (word_embeddings): Embedding(30523, 768)
        (position_embeddings): Embedding(512, 768)
        (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
        (dropout): Dropout(p=0.1, inplace=False)
      )
      (transformer): Transformer(
        (layer): ModuleList(
          (0-5): 6 x TransformerBlock(
            (attention): MultiHeadSelfAttention(
              (dropout): Dropout(p=0.1, inplace=False)
              (q_lin): Linear(in_features=768, out_features=768, bias=True)
              (k_lin): Linear(in_features=768, out_features=768, bias=True)
              (v_lin): Linear(in_features=768, out_features=768, bias=True)
              (out_lin): Linear(in_features=768, out

100%|██████████| 14/14 [00:08<00:00,  1.56it/s]

...

2023-10-16 12:19:28,531 Evaluating as a multi-label problem: False
2023-10-16 12:19:28,548 DEV : loss 0.5244603157043457 - f1-score (micro avg)  0.7739


2023-10-16 12:19:29,828 BAD EPOCHS (no improvement): 0
2023-10-16 12:19:29,834 saving best model
2023-10-16 12:19:31,539 ----------------------------------------------------------------------------------------------------


100%|██████████| 15/15 [00:11<00:00,  1.31it/s]

2023-10-16 12:19:44,620 Evaluating as a multi-label problem: False
2023-10-16 12:19:44,638 0.7437	0.7437	0.7437	0.7437
2023-10-16 12:19:44,640 
Results:
- F-score (micro) 0.7437
- F-score (macro) 0.7292
- Accuracy 0.7437

By class:
              precision    recall  f1-score   support

     Neutral     0.8406    0.7484    0.7918       310
  NotNeutral     0.6100    0.7349    0.6667       166

    accuracy                         0.7437       476
   macro avg     0.7253    0.7417    0.7292       476
weighted avg     0.7602    0.7437    0.7482       476

2023-10-16 12:19:44,642 ----------------------------------------------------------------------------------------------------


{'test_score': 0.7436974789915967,
 'dev_score_history': [0.7249417249417249,
  0.7435897435897436,
  0.7738927738927739],
 'train_loss_history': [0.6166844264159492,
  0.5267105292878416,
  0.4692852861711928],
 'dev_loss_history': [0.6055618524551392,
  0.5088042616844177,
  0.5244603157043457]}

Change the learning rate, keep training

In [ ]:
trainer.train(
    #'/content/gdrive/MyDrive/Enron_email/Flair/model_with_validation',  # Specify the path to save the model
    #'/content/gdrive/MyDrive/Enron_email/Flair/bert_model_final',
    #'/content/gdrive/MyDrive/Enron_email/Flair/model_aug',
    #'/content/gdrive/MyDrive/Enron_email/Flair/model_semi',
    '/content/gdrive/MyDrive/Enron_email/Flair/model1',

    learning_rate=0.005,
    mini_batch_size=32,
    # max_epochs=10,
    max_epochs = 3
)

2023-10-16 12:21:48,571 ----------------------------------------------------------------------------------------------------
2023-10-16 12:21:48,575 Model: "TextClassifier(
  (embeddings): TransformerDocumentEmbeddings(
    (model): DistilBertModel(
      (embeddings): Embeddings(
        (word_embeddings): Embedding(30523, 768)
        (position_embeddings): Embedding(512, 768)
        (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
        (dropout): Dropout(p=0.1, inplace=False)
      )
      (transformer): Transformer(
        (layer): ModuleList(
          (0-5): 6 x TransformerBlock(
            (attention): MultiHeadSelfAttention(
              (dropout): Dropout(p=0.1, inplace=False)
              (q_lin): Linear(in_features=768, out_features=768, bias=True)
              (k_lin): Linear(in_features=768, out_features=768, bias=True)
              (v_lin): Linear(in_features=768, out_features=768, bias=True)
              (out_lin): Linear(in_features=768, out

/usr/local/lib/python3.10/dist-packages/flair/trainers/trainer.py:69: UserWarning: There should be no best model saved at epoch 1 except there is a model from previous trainings in your training folder. All previous best models will be deleted.
  warnings.warn(


2023-10-16 12:22:05,841 epoch 1 - iter 12/121 - loss 0.35090006 - time (sec): 17.23 - samples/sec: 22.29 - lr: 0.005000
2023-10-16 12:22:24,943 epoch 1 - iter 24/121 - loss 0.37926348 - time (sec): 36.33 - samples/sec: 21.14 - lr: 0.005000
2023-10-16 12:22:43,656 epoch 1 - iter 36/121 - loss 0.36943935 - time (sec): 55.04 - samples/sec: 20.93 - lr: 0.005000
2023-10-16 12:23:01,955 epoch 1 - iter 48/121 - loss 0.35183496 - time (sec): 73.34 - samples/sec: 20.94 - lr: 0.005000
2023-10-16 12:23:19,853 epoch 1 - iter 60/121 - loss 0.35480740 - time (sec): 91.24 - samples/sec: 21.04 - lr: 0.005000
2023-10-16 12:23:36,641 epoch 1 - iter 72/121 - loss 0.35327815 - time (sec): 108.03 - samples/sec: 21.33 - lr: 0.005000
2023-10-16 12:23:54,461 epoch 1 - iter 84/121 - loss 0.35412722 - time (sec): 125.85 - samples/sec: 21.36 - lr: 0.005000
2023-10-16 12:24:12,683 epoch 1 - iter 96/121 - loss 0.35258946 - time (sec): 144.07 - samples/sec: 21.32 - lr: 0.005000
2023-10-16 12:24:30,783 epoch 1 - ite

100%|██████████| 14/14 [00:09<00:00,  1.52it/s]

2023-10-16 12:25:00,193 Evaluating as a multi-label problem: False
2023-10-16 12:25:00,209 DEV : loss 0.4818975031375885 - f1-score (micro avg)  0.7692


...

2023-10-16 12:31:31,322 Evaluating as a multi-label problem: False
2023-10-16 12:31:31,348 DEV : loss 0.5153331160545349 - f1-score (micro avg)  0.7692


2023-10-16 12:31:32,975 BAD EPOCHS (no improvement): 1
2023-10-16 12:31:33,889 ----------------------------------------------------------------------------------------------------


100%|██████████| 15/15 [00:10<00:00,  1.45it/s]

2023-10-16 12:31:45,867 Evaluating as a multi-label problem: False
2023-10-16 12:31:45,889 0.75	0.75	0.75	0.75
2023-10-16 12:31:45,893 
Results:
- F-score (micro) 0.75
- F-score (macro) 0.702
- Accuracy 0.75

By class:
              precision    recall  f1-score   support

     Neutral     0.7675    0.8839    0.8216       310
  NotNeutral     0.6975    0.5000    0.5825       166

    accuracy                         0.7500       476
   macro avg     0.7325    0.6919    0.7020       476
weighted avg     0.7431    0.7500    0.7382       476

2023-10-16 12:31:45,897 ----------------------------------------------------------------------------------------------------


{'test_score': 0.75,
 'dev_score_history': [0.7692307692307693,
  0.7715617715617715,
  0.7692307692307693],
 'train_loss_history': [0.398372258962035,
  0.3795696081059655,
  0.34550870113758425],
 'dev_loss_history': [0.4818975031375885,
  0.515893816947937,
  0.5153331160545349]}

lower the learning rate, Continue to train

In [ ]:
trainer.train(
    #'/content/gdrive/MyDrive/Enron_email/Flair/model_with_validation',  # Specify the path to save the model
    #'/content/gdrive/MyDrive/Enron_email/Flair/bert_model_final',
    #'/content/gdrive/MyDrive/Enron_email/Flair/model_aug',
    #'/content/gdrive/MyDrive/Enron_email/Flair/model_semi',
    '/content/gdrive/MyDrive/Enron_email/Flair/model1',

    learning_rate=0.0025,
    mini_batch_size=32,
    # max_epochs=10,
    max_epochs = 3
)

2023-10-17 04:07:13,035 Reading data from /content/gdrive/MyDrive/Enron_email/Flair/model1_train
2023-10-17 04:07:13,036 Train: /content/gdrive/MyDrive/Enron_email/Flair/model1_train/train.csv
2023-10-17 04:07:13,039 Dev: None
2023-10-17 04:07:13,042 Test: None


/usr/local/lib/python3.10/dist-packages/flair/trainers/trainer.py:69: UserWarning: There should be no best model saved at epoch 1 except there is a model from previous trainings in your training folder. All previous best models will be deleted.
  warnings.warn(


2023-10-17 04:07:14,753 ----------------------------------------------------------------------------------------------------
2023-10-17 04:07:14,758 Model: "TextClassifier(
  (embeddings): TransformerDocumentEmbeddings(
    (model): DistilBertModel(
      (embeddings): Embeddings(
        (word_embeddings): Embedding(30523, 768)
        (position_embeddings): Embedding(512, 768)
        (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
        (dropout): Dropout(p=0.1, inplace=False)
      )
      (transformer): Transformer(
        (layer): ModuleList(
          (0-5): 6 x TransformerBlock(
            (attention): MultiHeadSelfAttention(
              (dropout): Dropout(p=0.1, inplace=False)
              (q_lin): Linear(in_features=768, out_features=768, bias=True)
              (k_lin): Linear(in_features=768, out_features=768, bias=True)
              (v_lin): Linear(in_features=768, out_features=768, bias=True)
              (out_lin): Linear(in_features=768, out

100%|██████████| 14/14 [00:09<00:00,  1.54it/s]


2023-10-17 04:10:22,005 Evaluating as a multi-label problem: False
2023-10-17 04:10:22,034 DEV : loss 0.3692004382610321 - f1-score (micro avg)  0.8625
2023-10-17 04:10:23,017 BAD EPOCHS (no improvement): 0
2023-10-17 04:10:23,023 saving best model
2023-10-17 04:10:23,854 ----------------------------------------------------------------------------------------------------
2023-10-17 04:10:41,897 epoch 2 - iter 12/121 - loss 0.38377795 - time (sec): 18.04 - samples/sec: 21.29 - lr: 0.002500
2023-10-17 04:11:00,766 epoch 2 - iter 24/121 - loss 0.38446408 - time (sec): 36.91 - samples/sec: 20.81 - lr: 0.002500
2023-10-17 04:11:18,929 epoch 2 - iter 36/121 - loss 0.35389708 - time (sec): 55.07 - samples/sec: 20.92 - lr: 0.002500
2023-10-17 04:11:35,904 epoch 2 - iter 48/121 - loss 0.34686225 - time (sec): 72.05 - samples/sec: 21.32 - lr: 0.002500
2023-10-17 04:11:52,857 epoch 2 - iter 60/121 - loss 0.34767360 - time (sec): 89.00 - samples/sec: 21.57 - lr: 0.002500
2023-10-17 04:12:08,625 ep

...

2023-10-17 04:16:38,771 Evaluating as a multi-label problem: False
2023-10-17 04:16:38,789 DEV : loss 0.33177655935287476 - f1-score (micro avg)  0.8741


2023-10-17 04:16:39,613 BAD EPOCHS (no improvement): 1
2023-10-17 04:16:47,891 ----------------------------------------------------------------------------------------------------


100%|██████████| 15/15 [00:08<00:00,  1.70it/s]


2023-10-17 04:16:59,816 Evaluating as a multi-label problem: False
2023-10-17 04:16:59,849 0.8109	0.8109	0.8109	0.8109
2023-10-17 04:16:59,851 
Results:
- F-score (micro) 0.8109
- F-score (macro) 0.7952
- Accuracy 0.8109

By class:
              precision    recall  f1-score   support

     Neutral     0.8145    0.8931    0.8520       290
  NotNeutral     0.8038    0.6828    0.7384       186

    accuracy                         0.8109       476
   macro avg     0.8091    0.7879    0.7952       476
weighted avg     0.8103    0.8109    0.8076       476

2023-10-17 04:16:59,854 ----------------------------------------------------------------------------------------------------


{'test_score': 0.8109243697478992,
 'dev_score_history': [0.8624708624708625,
  0.8741258741258742,
  0.8741258741258742],
 'train_loss_history': [0.36040138246606457,
  0.34930577928257334,
  0.33109594343115223],
 'dev_loss_history': [0.3692004382610321,
  0.32578739523887634,
  0.33177655935287476]}

## Step 2: Delete runtime(Otherwise model will take too much memory, unable to run), then finetune a pre-trained model for Positive and Negative.

In [1]:
# import package again
import pandas as pd

# load google drive
from google.colab import drive
drive.mount('/content/gdrive')

# install Flair framework
! pip install flair

Mounted at /content/gdrive
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 373.1/373.1 kB 6.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 788.5/788.5 kB 34.2 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 135.8/135.8 kB 17.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.7/7.7 MB 83.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 981.5/981.5 kB 65.0 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 53.1/53.1 kB 6.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 19.7/19.7 MB 73.6 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 302.0/302.0 kB 28.1 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━

In [7]:
# load packages from Flair
from flair.models import TextClassifier
from flair.data import Sentence
from flair.data import Corpus
from flair.datasets import CSVClassificationCorpus
from flair.trainers import ModelTrainer

# Load the pre-trained sentiment classifier
classifier = TextClassifier.load('sentiment-fast')


In [8]:
# load training data
df = pd.read_csv('/content/gdrive/MyDrive/Enron_email/4763.csv')  # this time use oversampled dataset

df = df[['Content', 'Label']]  # subset the columns
df.columns = ['Text', 'Sentiment']   # change the column names

df = df[df['Sentiment'] != 'Neutral']  # get the Positive and Negative rows
df

,Text,Sentiment
3,I probably won't be able to go. Could we send ...,Negative
5,"Stevens, Joe put a call into Destin today, but...",Positive
6,We moved the meeting to 12:00 for tomorrow and...,Positive
8,thank ;you very much Kam Keiser 05/08/2001 01:...,Positive
10,Don't hate me for sending a note. What do you ...,Negative
...,...,...
4748,I don't know if I gave this to you yet. Sent: ...,Positive
4755,you're right about the email address. thanks. ...,Positive
4757,"see you there Sent: Thursday, September 27, 20...",Positive
4758,This is a summary of my position and trading g...,Positive


In [9]:
# convert labels to Capital because the pre-trained model out put capital letters
new_list = []  # create a list to store the labels

for index, row in df.iterrows():
  if row['Sentiment'] == 'Positive':
    new_list.append('POSITIVE')
  else:
    new_list.append('NEGATIVE')

df['Sentiment'] = new_list
df

,Text,Sentiment
3,I probably won't be able to go. Could we send ...,NEGATIVE
5,"Stevens, Joe put a call into Destin today, but...",POSITIVE
6,We moved the meeting to 12:00 for tomorrow and...,POSITIVE
8,thank ;you very much Kam Keiser 05/08/2001 01:...,POSITIVE
10,Don't hate me for sending a note. What do you ...,NEGATIVE
...,...,...
4748,I don't know if I gave this to you yet. Sent: ...,POSITIVE
4755,you're right about the email address. thanks. ...,POSITIVE
4757,"see you there Sent: Thursday, September 27, 20...",POSITIVE
4758,This is a summary of my position and trading g...,POSITIVE


In [10]:
df.to_csv('/content/gdrive/MyDrive/Enron_email/Flair/model2_train/train.csv',index = False)  # save into a train file

In [11]:
data_path = '/content/gdrive/MyDrive/Enron_email/Flair/model2_train'

# Define column names
column_names = {0: 'text', 1: 'label'}

# Create a Corpus using the CSVClassificationCorpus class
corpus: Corpus = CSVClassificationCorpus(data_folder = data_path,
                                         train_file = '/content/gdrive/MyDrive/Enron_email/Flair/model2_train/train.csv',
                                         column_name_map = {0: 'text', 1: 'label'}, label_type = 'Sentiment', skip_header=True)

trainer = ModelTrainer(classifier, corpus)

# Fine-tune the classifier
trainer.fine_tune(
    '/content/gdrive/MyDrive/Enron_email/Flair/model2',  # Specify the path to save the model, for the second model, we don't actually load model from this folder, we load saved state_dict
    learning_rate=0.01,
    mini_batch_size=32,
    max_epochs=20
)

2023-10-18 02:46:44,222 Reading data from /content/gdrive/MyDrive/Enron_email/Flair/model2_train
2023-10-18 02:46:44,224 Train: /content/gdrive/MyDrive/Enron_email/Flair/model2_train/train.csv
2023-10-18 02:46:44,226 Dev: None
2023-10-18 02:46:44,229 Test: None
2023-10-18 02:46:44,281 ----------------------------------------------------------------------------------------------------
2023-10-18 02:46:44,282 Model: "TextClassifier(
  (embeddings): DocumentRNNEmbeddings(
    (embeddings): StackedEmbeddings(
      (list_embedding_0): WordEmbeddings('en-crawl')
    )
    (word_reprojection_map): Linear(in_features=300, out_features=300, bias=True)
    (rnn): LSTM(300, 256, batch_first=True)
    (dropout): Dropout(p=0.5, inplace=False)
  )
  (decoder): Linear(in_features=256, out_features=2, bias=True)
  (dropout): Dropout(p=0.0, inplace=False)
  (locked_dropout): LockedDropout(p=0.0)
  (word_dropout): WordDropout(p=0.0)
  (loss_function): BCEWithLogitsLoss()
  (weights): None
  (weight_ten

100%|██████████| 5/5 [00:01<00:00,  3.02it/s]

2023-10-18 02:46:58,721 Evaluating as a multi-label problem: True


2023-10-18 02:46:58,802 DEV : loss 0.6910243630409241 - f1-score (micro avg)  0.9415
2023-10-18 02:46:59,339 ----------------------------------------------------------------------------------------------------
2023-10-18 02:47:00,494 epoch 2 - iter 4/43 - loss 0.90489863 - time (sec): 1.15 - samples/sec: 111.49 - lr: 0.005402
2023-10-18 02:47:01,510 epoch 2 - iter 8/43 - loss 0.90634547 - time (sec): 2.16 - samples/sec: 118.30 - lr: 0.005862
2023-10-18 02:47:02,160 epoch 2 - iter 12/43 - loss 0.89567632 - time (sec): 2.81 - samples/sec: 136.46 - lr: 0.006322
2023-10-18 02:47:03,381 epoch 2 - iter 16/43 - loss 0.91058319 - time (sec): 4.04 - samples/sec: 126.88 - lr: 0.006782
2023-10-18 02:47:04,128 epoch 2 - iter 20/43 - loss 0.88724230 - time (sec): 4.78 - samples/sec: 133.82 - lr: 0.007241
2023-10-18 02:47:05,008 epoch 2 - iter 24/43 - loss 0.88022889 - time (sec): 5.66 - samples/sec: 135.63 - lr: 0.007701
2023-10-18 02:47:05,657 epoch 2 - iter 28/43 - loss 0.90796150 - time (sec): 6

100%|██████████| 5/5 [00:00<00:00,  8.54it/s]

...

2023-10-18 02:50:42,865 Evaluating as a multi-label problem: True


2023-10-18 02:50:42,908 DEV : loss 0.7614547610282898 - f1-score (micro avg)  0.9273
2023-10-18 02:50:44,653 ----------------------------------------------------------------------------------------------------
2023-10-18 02:50:44,674 Testing using last state of model ...


100%|██████████| 6/6 [00:00<00:00,  6.60it/s]

2023-10-18 02:50:45,600 Evaluating as a multi-label problem: True
2023-10-18 02:50:45,622 1.0	0.8293	0.9067	0.7941
2023-10-18 02:50:45,625 
Results:
- F-score (micro) 0.9067
- F-score (macro) 0.8828
- Accuracy 0.7941

By class:
              precision    recall  f1-score   support

    POSITIVE     1.0000    0.8865    0.9398       141
    NEGATIVE     1.0000    0.7031    0.8257        64

   micro avg     1.0000    0.8293    0.9067       205
   macro avg     1.0000    0.7948    0.8828       205
weighted avg     1.0000    0.8293    0.9042       205
 samples avg     1.0000    0.8971    0.9314       205

2023-10-18 02:50:45,628 ----------------------------------------------------------------------------------------------------


{'test_score': 0.9066666666666667,
 'dev_score_history': [0.9415384615384615,
  0.8973607038123167,
  0.9216867469879518,
  0.9161676646706587,
  0.9244712990936556,
  0.9189189189189189,
  0.9161676646706587,
  0.9300911854103344,
  0.9300911854103344,
  0.9300911854103344,
  0.9080118694362018,
  0.8973607038123167,
  0.9244712990936556,
  0.9216867469879518,
  0.9329268292682927,
  0.9415384615384615,
  0.9329268292682927,
  0.9134328358208955,
  0.9386503067484663,
  0.9272727272727274],
 'train_loss_history': [1.1403793678283691,
  0.9266684126420455,
  1.0422619941017846,
  1.0238450414484197,
  0.935342639576305,
  0.938274150501598,
  0.96972036188299,
  0.8372741394042968,
  0.8466119634454901,
  0.8616214710582386,
  0.8271330608021129,
  0.8514165399724787,
  0.7922810960249468,
  0.7517330800836737,
  0.7329854625355113,
  0.7305406077991833,
  0.7255179748535157,
  0.6846445645419034,
  0.7000734017112038,
  0.618255593733354],
 'dev_loss_history': [0.6910243630409241,
  0

In [12]:
import torch
torch.save(classifier.state_dict(), "/content/gdrive/MyDrive/Enron_email/Flair/model2/saved_model")  # save the state_dict

## Step 3 Delete runtime, then testing

In [1]:
# import packages
import pandas as pd
from sklearn.metrics import accuracy_score, classification_report

In [2]:
# load google drive
from google.colab import drive
drive.mount('/content/gdrive')

Mounted at /content/gdrive


In [3]:
# install Flair framework
! pip install flair

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 373.1/373.1 kB 4.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 788.5/788.5 kB 11.1 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 135.8/135.8 kB 8.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.7/7.7 MB 30.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 981.5/981.5 kB 47.7 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 53.1/53.1 kB 6.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 19.7/19.7 MB 59.1 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 302.0/302.0 kB 32.8 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.

In [4]:
# load packages from Flair
from flair.models import TextClassifier
from flair.data import Sentence

In [5]:
# load the first model for predict Neutral and nonNeutral class
classifier1 = TextClassifier.load('/content/gdrive/MyDrive/Enron_email/Flair/model1/best-model.pt')

In [6]:
# import package
import torch

# load the second model for predict Positive and Negative
classifier2 = TextClassifier.load('sentiment-fast')

# load state_dict for the second model
classifier2.load_state_dict(torch.load("/content/gdrive/MyDrive/Enron_email/Flair/model2/saved_model"))
classifier2.eval()

2023-10-18 03:02:37,638 https://nlp.informatik.hu-berlin.de/resources/models/sentiment-curated-fasttext-rnn/sentiment-en-mix-ft-rnn_v8.pt not found in cache, downloading to /tmp/tmpkl2q3vvy


100%|██████████| 1.16G/1.16G [02:49<00:00, 7.32MB/s]

2023-10-18 03:05:28,004 copying /tmp/tmpkl2q3vvy to cache at /root/.flair/models/sentiment-en-mix-ft-rnn_v8.pt


2023-10-18 03:05:32,518 removing temp file /tmp/tmpkl2q3vvy


TextClassifier(
  (embeddings): DocumentRNNEmbeddings(
    (embeddings): StackedEmbeddings(
      (list_embedding_0): WordEmbeddings('en-crawl')
    )
    (word_reprojection_map): Linear(in_features=300, out_features=300, bias=True)
    (rnn): LSTM(300, 256, batch_first=True)
    (dropout): Dropout(p=0.5, inplace=False)
  )
  (decoder): Linear(in_features=256, out_features=2, bias=True)
  (dropout): Dropout(p=0.0, inplace=False)
  (locked_dropout): LockedDropout(p=0.0)
  (word_dropout): WordDropout(p=0.0)
  (loss_function): BCEWithLogitsLoss()
)

In [7]:
# load test set
df = pd.read_csv('/content/gdrive/MyDrive/Enron_email/1191.csv')

In [8]:
df = df[['Content', 'Label']]  # subset the columns
df.columns = ['Text', 'Sentiment']   # change the column names

In [9]:
# load torch
import torch
# Set device to GPU for fast prediction
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print(device)

# define a function for prediction
def get_sentiment(text):
    value = ''
    # Create a Sentence object
    sentence1 = Sentence(text)   #  create a sentence object for prediction

    # Predict the sentiment
    with torch.no_grad():
        classifier1.predict(sentence1)

    # Get the predicted label, if the prediction is Neutral, return the result, if the prediction is not Neutral, use the second model
    label1 = sentence1.labels[0]   # predicted label
    if label1.value == 'Neutral':
        return 'Neutral'
    else:

        sentence2 = Sentence(text)  # create a new sentence object, because the first one will contain the prediction of the first model

    # Predict the sentiment
        with torch.no_grad():
            classifier2.predict(sentence2)

    # Get the predicted label
        label2 = sentence2.labels[0]   # predicted label

        return label2.value.title()   # title case the prediction, because the second model only out put capital letters


cuda


In [10]:
df['result'] = df['Text'].apply(get_sentiment)   # add prediction result column

In [11]:
predicted_labels = df['result']
actual_labels = df['Sentiment']

# Calculate accuracy overall
accuracy = accuracy_score(actual_labels, predicted_labels)
print(f'Overall Accuracy: {accuracy:.2f}')

# Calculate accuracy and f1 scores by class
class_report = classification_report(actual_labels, predicted_labels)
print(class_report)

Overall Accuracy: 0.76
              precision    recall  f1-score   support

    Negative       0.47      0.44      0.46       104
     Neutral       0.81      0.88      0.84       764
    Positive       0.71      0.58      0.64       323

    accuracy                           0.76      1191
   macro avg       0.66      0.63      0.65      1191
weighted avg       0.75      0.76      0.75      1191

